In [72]:
import altair as alt
import pandas as pd
# success_table = pd.read_csv('../fig_02/figure2_input.tsv', sep='\t')
# success_table = pd.read_csv('figure3_not_prior.tsv', sep='\t')
success_table = pd.read_csv('GS_exomiser_fig3_input_12.15.24.tsv', sep='\t')
import numpy as np


In [1]:
import sys
sys.path.append('figure_scripts/')
 
import plot_scripts as ps

### A: Different combinations of pathogenicity predictors

In [76]:
fig_A = ['noN_filtered_15_85_human_revel_mvp_noWL','noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL', 'noN_filtered_15_85_human_polyphen_mutTaster_sift_noWL','noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_polyphen_mutTaster_sift_noWL', 'noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL']
combo_domain = ['noN_filtered_15_85_human_revel_mvp_noWL','noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL','noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL', 'noN_filtered_15_85_human_polyphen_mutTaster_sift_noWL','noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_polyphen_mutTaster_sift_noWL']

A_df,denom = ps.create_df(535, fig_A, success_table)
plot_A=ps.create_plot(denom, A_df, fig_A, 'set2', False,combo_domain)

plot_A.configure_axis(grid=True,
    labelPadding= 5,
    labelLimit=0,
    labelFontSize=12, 
    titleFontSize=15, labelFont='arial', tickSize=8).configure_legend(
        labelLimit=0,labelFontSize=15, titleFontSize=15, labelFont='arial').properties(height=300, width=500)

highest rank: 277.0 max%: 96.95945945945947 287
highest rank: 78.0 max%: 97.2972972972973 288
highest rank: 120.0 max%: 96.95945945945947 287
highest rank: 120.0 max%: 96.95945945945947 287
highest rank: 147.0 max%: 96.62162162162163 286


### B-C: Distribution of MAX_PATH_SOURCE

In [85]:
all_results = pd.read_csv('GS_all_results_path_source_combos.tsv', sep='\t')
order=['NONE', 'ALPHA_MISSENSE', 'MVP', 'REVEL', 'SPLICE_AI', 'CADD','MUTATION_TASTER', 'POLYPHEN', 'SIFT','REMM', 'Not_Prioritized']
RUN_TYPES = ['noN_filtered_15_85_human_revel_mvp_noWL','noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL', 'noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL','noN_filtered_15_85_human_polyphen_mutTaster_sift_noWL','noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_polyphen_mutTaster_sift_noWL']


all_results['variant'] = [x.split('_')[0] for x in all_results['ID']]
success_table['variant'] = [x.split('_')[0] for x in success_table['variant_exomiser_form']]
all_results['id_variant'] = np.array(all_results['Patient_ID']) + '_' + np.array(all_results['variant'])
success_table['UDN_ID'] =[x.split('_')[0] for x in list(success_table['ID'])]
success_table['id_variant'] = np.array(success_table['UDN_ID']) + '_' + np.array(success_table['variant'])
all_results.fillna('NONE', inplace=True)


/tmp/ipykernel_504431/4130562706.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NONE' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_results.fillna('NONE', inplace=True)


In [86]:
non_diagnostic = all_results[~all_results['id_variant'].isin(list(success_table['id_variant']))]
diagnostic = all_results[all_results['id_variant'].isin(list(success_table['id_variant']))]


In [87]:
data = []
for RUN_TYPE in RUN_TYPES:
    #results = all_results[all_results['RunType']==RUN_TYPE]
    results = non_diagnostic[non_diagnostic['RunType']==RUN_TYPE]
    results_noNone = results[results['MAX_PATH'] != 'NONE']
    print(RUN_TYPE, len(results_noNone))

    for path_source in order:
        path = results_noNone[results_noNone['MAX_PATH_SOURCE']==path_source]
      #  print(path_source, len(path))
        data.append([RUN_TYPE, path_source, len(path), len(path)/len(results_noNone), len(results)])
        df = pd.DataFrame(data, columns=['RunType', 'MAX_PATH_SOURCE', 'num', 'percentage', 'all_runtype'])
        
base=alt.Chart(df,title='Max path sources for NONDIAGNOSTIC variants')       
bars = base.mark_bar().encode(
    alt.X('RunType:N', sort=RUN_TYPES),
    alt.Y('num:Q', title='Number of variants'),
    color=alt.Color('MAX_PATH_SOURCE:N', sort=order, scale=alt.Scale(scheme='tableau20', domain=order)),
    tooltip=['RunType:N','MAX_PATH_SOURCE', 'num:Q', 'percentage']

).properties(width=400, height=200)
text = base.mark_point(
    align='left',
    baseline='middle',
    #dx=-5,
    #dy=-10,
    strokeDash=[5,1],
    size=5000
  
).encode(
    y=alt.Y('all_runtype:Q').axis(offset=5, domainOpacity=0),
    x=alt.X('RunType:N', sort=RUN_TYPES), 
    color=alt.value('black'),
    shape=alt.Shape('RunType:N',scale = alt.Scale(range=["stroke"])),
    text='all_runtype:Q'
)
all_variants = alt.layer(bars, text).resolve_scale(color='independent')#.properties( width=400).configure_axis(grid=False,
    # labelPadding= 5,
    # labelLimit=0,
    # labelFontSize=15, 
    # titleFontSize=15, labelFont='arial', tickSize=8).configure_legend(
    #     labelLimit=0,labelFontSize=15, titleFontSize=15, labelFont='arial')




noN_filtered_15_85_human_revel_mvp_noWL 27039
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL 34517
noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL 61308
noN_filtered_15_85_human_polyphen_mutTaster_sift_noWL 29189
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_polyphen_mutTaster_sift_noWL 36763


In [88]:
data = []
for RUN_TYPE in RUN_TYPES:
    results = success_table[success_table['Variant_Level_noMOI_' + str(RUN_TYPE)]=='Variant_Present_noMOI']
    results_noNone = results[results['Variant_Level_noMOI_maxPathSource_'+str(RUN_TYPE)] != 'NONE']
    max_paths = list(results_noNone['Variant_Level_noMOI_maxPathSource_'+str(RUN_TYPE)])
    print(RUN_TYPE, len(results))

    for path_source in order:
        num_path = max_paths.count(path_source)
      #  print(path_source, len(path))
        data.append([RUN_TYPE, path_source, num_path, num_path/len(max_paths), len(results)])

df = pd.DataFrame(data, columns=['RunType', 'MAX_PATH_SOURCE', 'num', 'percentage', 'all_runtype'])

base = alt.Chart(df,title='Max path sources for DIAGNOSTIC variants')
bars =base.mark_bar().encode(
    alt.X('RunType:N', sort=RUN_TYPES),
    alt.Y('num:Q', title='Number of variants (max=296)'),
    color=alt.Color('MAX_PATH_SOURCE:N', sort=order, scale=alt.Scale(scheme='tableau20', domain=order))

).properties(width=400, height=200)
text = base.mark_point(
    align='left',
    baseline='middle',
    dx=-5,
    dy=-10,
    color='black',
    strokeDash=[5,1],
    size=5000
).encode(
    y=alt.Y('all_runtype:Q').axis(offset=5, domainOpacity=0),
    x=alt.X('RunType:N', sort=RUN_TYPES), 
    color=alt.value('black'),
    shape=alt.Shape('RunType:N',scale = alt.Scale(range=["stroke"])),
    text='all_runtype:Q'
)

text2 = base.mark_text(
    align='left',
    baseline='middle',
    dx=-5,
    dy=-5,
    color='black',

).encode(
    y=alt.Y('all_runtype:Q').axis(offset=5, domainOpacity=0),
    x=alt.X('RunType:N', sort=RUN_TYPES), 
    color=alt.value('black'),
    text='all_runtype:Q'
)

diagnostic_variants = alt.layer(bars, text).resolve_scale(color='independent')#.properties( width=400).configure_axis(grid=False,
    # labelPadding= 5,
    # labelLimit=0,
    # labelFontSize=15, 
    # titleFontSize=15, labelFont='arial', tickSize=8).configure_legend(
    #     labelLimit=0,labelFontSize=15, titleFontSize=15, labelFont='arial')

noN_filtered_15_85_human_revel_mvp_noWL 287
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL 288
noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL 286
noN_filtered_15_85_human_polyphen_mutTaster_sift_noWL 287
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_polyphen_mutTaster_sift_noWL 287


In [89]:
variant_bars = alt.vconcat(all_variants, diagnostic_variants)
variant_bars

alt.VConcatChart(...)

### D-E: Distribution of maximum pathogenicity scores

In [90]:
non_diagnostic = all_results[~all_results['id_variant'].isin(list(success_table['id_variant']))]

In [91]:
from pathlib import Path

tempdir = Path("./temp")
tempdir.mkdir(exist_ok=True)

source = tempdir / "data.json"
non_diagnostic[~non_diagnostic['MAX_PATH'].isin(['NONE' , 'Not_Prioritized'])].to_json(source, orient="records")

In [92]:
for run_type in RUN_TYPES:
    scores = non_diagnostic[non_diagnostic['RunType']==run_type]
    scores = scores[scores['MAX_PATH']!='NONE']
    print(run_type,len(scores))

noN_filtered_15_85_human_revel_mvp_noWL 27039
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL 34517
noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL 61308
noN_filtered_15_85_human_polyphen_mutTaster_sift_noWL 29189
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_polyphen_mutTaster_sift_noWL 36763


In [94]:

all_variants_max_path =alt.Chart(str(source), title='Variant path scores of NONDIAGNOSTIC variants').mark_boxplot().encode(
    alt.X("MAX_PATH:Q").scale(zero=False),
    alt.Y('RunType:N',sort=combo_domain,title=None),#,axis=alt.Axis(labels=False)),
    color=alt.Color('RunType:N', scale=alt.Scale(scheme='set2', domain=combo_domain))).properties(width=500)

In [97]:
data = []
for run_type in RUN_TYPES:
    variant_scores = list(success_table['Variant_Level_noMOI_variant_'+run_type])
    for score in variant_scores:
        data.append([score, run_type])
df= pd.DataFrame(data, columns=['EXOMISER_VARIANT_SCORE', 'RunType'])


data2 = []
for run_type in RUN_TYPES:
    variant_scores = list(success_table['Variant_Level_noMOI_maxPath_'+run_type])
    for score in variant_scores:
        data2.append([score, run_type])
df2= pd.DataFrame(data2, columns=['MAX_PATH', 'RunType'])

In [98]:


diagnostic_only_maxpath =alt.Chart(df2, title='Pathogenicity score of DIAGNOSTIC variants only').mark_boxplot().encode(
    alt.X("MAX_PATH:Q").scale(zero=False),
    alt.Y('RunType:N',sort=combo_domain,title=None),
    color=alt.Color('RunType:N', scale=alt.Scale(scheme='set2', domain=combo_domain))).properties(width=500)

In [100]:
right_panel = alt.vconcat(diagnostic_variants, all_variants)
left_panel = alt.vconcat(fig_A,all_variants_max_path, diagnostic_only_maxpath)
alt.hconcat(left_panel, right_panel).configure_axis(grid=True,
    labelPadding= 5,
    labelLimit=0,
    labelFontSize=15, 
    titleFontSize=15, labelFont='arial', tickSize=8).configure_legend(
        labelLimit=0,labelFontSize=15, titleFontSize=15, labelFont='arial')

alt.HConcatChart(...)